In [1]:
!pip install plac
!pip install spacy
!python -m spacy download en_core_web_md

     |████████████████████████████████| 45.7 MB 26.4 MB/s eta 0:00:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [2]:
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.training import Example
import srsly
import pickle
import os
import csv
import numpy as np
import time

In [3]:
# creating a folder to hold the data files, model results, output etc.
model_dir_name = "custom_entity_extractor_spacy_change_direction_test"
model_dir_parent = "/Users/ash/Desktop"
output_dir = os.path.join(model_dir_parent, model_dir_name)
os.makedirs(output_dir, exist_ok=True)
print("created", output_dir)

created /Users/ash/Desktop/custom_entity_extractor_spacy_change_direction_test


In [4]:
# below code is to convert a jsonl file in this format to spacy format:
# {"order":10,"Used?":"staged","Dataset name":"checkin-three","ignore?":"","reason":"","relation count":"many","complexity":"complex","original_md_text":"We also know that emotional distress during pregnancy and mood disorders, which can be triggered as women face the uncertainty that climate change brings to their lives and the lives of their families, can affect not only the physical but also the neurological and cognitive development of the unborn child.\n","text":"We also know that emotional distress during pregnancy and mood disorders, which can be triggered as women face the uncertainty that climate change brings to their lives and the lives of their families, can affect not only the physical but also the neurological and cognitive development of the unborn child. IMPLIED_BASE IMPLIED_BASE","characters":308,"document_index":14,"md_sentence_index":35,"classification_origin":"cm-label-eval","url":"https://www.climaterealityproject.org/blog/dose-reality-how-climate-change-affects-our-kids-straight-pediatrician","_input_hash":690545709,"_task_hash":1928498526,"spans":[{"start":8,"end":12,"token_start":2,"token_end":2,"label":"confidence"},{"start":18,"end":27,"token_start":4,"token_end":4,"label":"type_of"},{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},{"start":37,"end":53,"token_start":6,"token_end":7,"label":"when"},{"start":58,"end":62,"token_start":9,"token_end":9,"label":"type_of"},{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},{"start":80,"end":83,"token_start":13,"token_end":13,"label":"confidence"},{"start":87,"end":96,"token_start":15,"token_end":15,"label":"change_direction"},{"start":97,"end":99,"token_start":16,"token_end":16,"label":"predicate"},{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},{"start":132,"end":146,"token_start":22,"token_end":23,"label":"base"},{"start":147,"end":153,"token_start":24,"token_end":24,"label":"predicate"},{"start":192,"end":200,"token_start":33,"token_end":33,"label":"to_whom"},{"start":202,"end":205,"token_start":35,"token_end":35,"label":"confidence"},{"start":206,"end":212,"token_start":36,"token_end":36,"label":"predicate"},{"start":226,"end":234,"token_start":40,"token_end":40,"label":"type_of"},{"start":248,"end":260,"token_start":44,"token_end":44,"label":"type_of"},{"start":265,"end":274,"token_start":46,"token_end":46,"label":"type_of"},{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"}],"tokens":[{"text":"We","start":0,"end":2,"id":0,"ws":true,"disabled":false},{"text":"also","start":3,"end":7,"id":1,"ws":true,"disabled":false},{"text":"know","start":8,"end":12,"id":2,"ws":true,"disabled":false},{"text":"that","start":13,"end":17,"id":3,"ws":true,"disabled":false},{"text":"emotional","start":18,"end":27,"id":4,"ws":true,"disabled":false},{"text":"distress","start":28,"end":36,"id":5,"ws":true,"disabled":false},{"text":"during","start":37,"end":43,"id":6,"ws":true,"disabled":false},{"text":"pregnancy","start":44,"end":53,"id":7,"ws":true,"disabled":false},{"text":"and","start":54,"end":57,"id":8,"ws":true,"disabled":false},{"text":"mood","start":58,"end":62,"id":9,"ws":true,"disabled":false},{"text":"disorders","start":63,"end":72,"id":10,"ws":false,"disabled":false},{"text":",","start":72,"end":73,"id":11,"ws":true,"disabled":false},{"text":"which","start":74,"end":79,"id":12,"ws":true,"disabled":false},{"text":"can","start":80,"end":83,"id":13,"ws":true,"disabled":false},{"text":"be","start":84,"end":86,"id":14,"ws":true,"disabled":false},{"text":"triggered","start":87,"end":96,"id":15,"ws":true,"disabled":false},{"text":"as","start":97,"end":99,"id":16,"ws":true,"disabled":false},{"text":"women","start":100,"end":105,"id":17,"ws":true,"disabled":false},{"text":"face","start":106,"end":110,"id":18,"ws":true,"disabled":false},{"text":"the","start":111,"end":114,"id":19,"ws":true,"disabled":false},{"text":"uncertainty","start":115,"end":126,"id":20,"ws":true,"disabled":false},{"text":"that","start":127,"end":131,"id":21,"ws":true,"disabled":false},{"text":"climate","start":132,"end":139,"id":22,"ws":true,"disabled":false},{"text":"change","start":140,"end":146,"id":23,"ws":true,"disabled":false},{"text":"brings","start":147,"end":153,"id":24,"ws":true,"disabled":false},{"text":"to","start":154,"end":156,"id":25,"ws":true,"disabled":false},{"text":"their","start":157,"end":162,"id":26,"ws":true,"disabled":false},{"text":"lives","start":163,"end":168,"id":27,"ws":true,"disabled":false},{"text":"and","start":169,"end":172,"id":28,"ws":true,"disabled":false},{"text":"the","start":173,"end":176,"id":29,"ws":true,"disabled":false},{"text":"lives","start":177,"end":182,"id":30,"ws":true,"disabled":false},{"text":"of","start":183,"end":185,"id":31,"ws":true,"disabled":false},{"text":"their","start":186,"end":191,"id":32,"ws":true,"disabled":false},{"text":"families","start":192,"end":200,"id":33,"ws":false,"disabled":false},{"text":",","start":200,"end":201,"id":34,"ws":true,"disabled":false},{"text":"can","start":202,"end":205,"id":35,"ws":true,"disabled":false},{"text":"affect","start":206,"end":212,"id":36,"ws":true,"disabled":false},{"text":"not","start":213,"end":216,"id":37,"ws":true,"disabled":false},{"text":"only","start":217,"end":221,"id":38,"ws":true,"disabled":false},{"text":"the","start":222,"end":225,"id":39,"ws":true,"disabled":false},{"text":"physical","start":226,"end":234,"id":40,"ws":true,"disabled":false},{"text":"but","start":235,"end":238,"id":41,"ws":true,"disabled":false},{"text":"also","start":239,"end":243,"id":42,"ws":true,"disabled":false},{"text":"the","start":244,"end":247,"id":43,"ws":true,"disabled":false},{"text":"neurological","start":248,"end":260,"id":44,"ws":true,"disabled":false},{"text":"and","start":261,"end":264,"id":45,"ws":true,"disabled":false},{"text":"cognitive","start":265,"end":274,"id":46,"ws":true,"disabled":false},{"text":"development","start":275,"end":286,"id":47,"ws":true,"disabled":false},{"text":"of","start":287,"end":289,"id":48,"ws":true,"disabled":false},{"text":"the","start":290,"end":293,"id":49,"ws":true,"disabled":false},{"text":"unborn","start":294,"end":300,"id":50,"ws":true,"disabled":false},{"text":"child","start":301,"end":306,"id":51,"ws":false,"disabled":false},{"text":".","start":306,"end":307,"id":52,"ws":true,"disabled":false},{"text":"IMPLIED_BASE","start":308,"end":320,"id":53,"ws":true,"disabled":false},{"text":"IMPLIED_BASE","start":321,"end":333,"id":54,"ws":false,"disabled":false}],"_session_id":"checkin_three_all_labels-answers","_view_id":"relations","relations":[{"head":4,"child":5,"head_span":{"start":18,"end":27,"token_start":4,"token_end":4,"label":"type_of"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":9,"child":10,"head_span":{"start":58,"end":62,"token_start":9,"token_end":9,"label":"type_of"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":46,"child":51,"head_span":{"start":265,"end":274,"token_start":46,"token_end":46,"label":"type_of"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":44,"child":51,"head_span":{"start":248,"end":260,"token_start":44,"token_end":44,"label":"type_of"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":40,"child":51,"head_span":{"start":226,"end":234,"token_start":40,"token_end":40,"label":"type_of"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":7,"child":5,"head_span":{"start":37,"end":53,"token_start":6,"token_end":7,"label":"when"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":7,"child":10,"head_span":{"start":37,"end":53,"token_start":6,"token_end":7,"label":"when"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":15,"child":10,"head_span":{"start":87,"end":96,"token_start":15,"token_end":15,"label":"change_direction"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":15,"child":5,"head_span":{"start":87,"end":96,"token_start":15,"token_end":15,"label":"change_direction"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":17,"child":5,"head_span":{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":17,"child":10,"head_span":{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":17,"child":20,"head_span":{"start":100,"end":105,"token_start":17,"token_end":17,"label":"to_whom"},"child_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":33,"child":20,"head_span":{"start":192,"end":200,"token_start":33,"token_end":33,"label":"to_whom"},"child_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"color":"#c5bdf4","label":"Concept_Member"},{"head":23,"child":20,"head_span":{"start":132,"end":146,"token_start":22,"token_end":23,"label":"base"},"child_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":20,"child":5,"head_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"child_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":20,"child":10,"head_span":{"start":115,"end":126,"token_start":20,"token_end":20,"label":"base"},"child_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":5,"child":51,"head_span":{"start":28,"end":36,"token_start":5,"token_end":5,"label":"base"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#ffd882","label":"Contributes_To"},{"head":10,"child":51,"head_span":{"start":63,"end":72,"token_start":10,"token_end":10,"label":"base"},"child_span":{"start":275,"end":306,"token_start":47,"token_end":51,"label":"base"},"color":"#ffd882","label":"Contributes_To"}],"answer":"accept"}
# above is only one entry from the jsonl file

In [5]:
# Create Training data
# 70% training data and 30% validation/testing data

# spacy requires data to be in a specific format which is what the code below does. Here is the acceptable formart:
# [('Heat stress and persistent dehydration can cause kidney damage. IMPLIED_BASE IMPLIED_BASE', {'entities': [(5, 11, 'base'), (27, 38, 'base'), (49, 55, 'base')]})]

file_name_answers = "main_3_per_cluster_download.66081fcf-3ef5-48ea-97e0-49298d29b477"
file_path_answers = "/Users/ash/Desktop/" + file_name_answers + ".jsonl"
data = srsly.read_jsonl(file_path_answers)
final_sent = []

for entry in data:
    if "text" in entry:
        text = entry["text"]
    label_arr = []
    label_tup = ()
    if entry['answer'] == "accept":
        if entry['_session_id'] == "main_3_per_cluster-Kameron":
            for relation in entry['spans']:
                if ("label" in relation) and ("start" in relation) and ("end" in relation):
                    child_span_start = relation["start"]
                    child_span_end = relation["end"]
                    word = text[child_span_start:child_span_end]
                    if relation["label"] == "change_direction":
                        tmp_tuple = (child_span_start, child_span_end, relation["label"])
                        label_arr.append(tmp_tuple)
            if len(label_arr) > 0:
                label_tup = (text, {"entities": label_arr})
                final_sent.append(label_tup)
            else:
                print(text)
                
training_data_index = int(len(final_sent)*0.7)

with open(os.path.join(output_dir, 'training_data.txt'), 'wb') as file:
    pickle.dump(final_sent[:training_data_index], file)
    
with open(os.path.join(output_dir, 'testing_data.txt'), 'wb') as file:
    pickle.dump(final_sent[training_data_index:], file)

(Later, a severe mid-summer Arctic heat wave contributed to historic melting of the Greenland ice sheet, with 12.5 billion tons of ice melting into the ocean on a single day — the “biggest single-day volume loss on record,” according to the Washington Post). IMPLIED_BASE IMPLIED_BASE
(Nitrous Oxide)—A clear non-flammable gas that has 300 times the global warming power of CO2; the majority of the N2O emissions are man-made n NOX (Nitrogen Dioxide and Nitric Acid)—Gases from the combustion of fossil fuels; react in the atmosphere to form O3, PM2 .5, and PM10 n Pb (Lead)—Elemental particulate coming primarily from leaded fuel n PM2.5—Particles with a diameter of 2 .5 micrometers or less coming from dust, smoke and soot, or formed from chemical reactions in the atmosphere n PM10—Particles with a diameter of 10 micrometers or less (includes PM2 .5) IMPLIED_BASE IMPLIED_BASE
For the research, Lo and her collaborators focused on so-called “1-in-30 events,” severe heat waves that strike every 

In [6]:
LABEL = ['change_direction']

with open (os.path.join(output_dir, 'training_data.txt'), 'rb') as file:
    TRAIN_DATA = pickle.load(file)

with open (os.path.join(output_dir, 'testing_data.txt'), 'rb') as file:
    TEST_DATA = pickle.load(file)

In [7]:
model = "en_core_web_md"
new_model_name = "custom_entity_extraction_change_dir"
n_iter = 100
batch_size = 50

In [8]:
print(len(TRAIN_DATA))
print(len(TEST_DATA))

174
75


In [9]:
TRAIN_DATA

[('These risks are crop insurance, health care, wildfire suppression, hurricane-related disaster relief, and federal facility flood risk, all of which are anticipated to cost billions of dollars more by the end of the century due to the impacts of climate change.23 IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(192, 196, 'change_direction')]}),
 ('The IPCC reported that Alexandria’s beaches would be submerged even with a 0.5-metre sea-level rise, while 8 million people would be displaced by flooding in Alexandria and the Nile Delta if no protective measures are taken. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(95, 99, 'change_direction'),
    (133, 142, 'change_direction')]}),
 ('One is warming that leads to a rain-on-snow event, which can cause flooding and flushing of nutrients out of the system, and another is a winter heat wave, such as one that hit New England in February 2017 with 70-degree temperatures. IMPLIED_BASE IMPLIED_BASE',
  {'entities': [(7, 14, 'change_direction'), (80

In [10]:
print("starting time: " + time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(time.time())))
if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')
    print("Created blank 'en' model")
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

for i in LABEL:
    # Adding new entity
    ner.add_label(i)

if model is None:
    optimizer = nlp.begin_training()
else:
    #optimizer = nlp.entity.create_optimizer()
    optimizer = nlp.create_optimizer()
    
# Get names of other pipes to disable them during training to train only NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        #for batch in batches:
        #    texts, annotations = zip(*batch)
        #    nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        #print('Losses', losses)
        for batch in spacy.util.minibatch(TRAIN_DATA, size=batch_size):
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
        print(str(itn) + " loss: ", losses)
        
# Save model 
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

print("ending time: " + time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(time.time())))

starting time: 03/26/2022 14:42:26
Loaded model 'en_core_web_md'
0 loss:  {'ner': 651.9750212941941}
1 loss:  {'ner': 420.3180984003043}
2 loss:  {'ner': 338.6103218876556}
3 loss:  {'ner': 277.96610247956943}
4 loss:  {'ner': 268.9508795330492}
5 loss:  {'ner': 222.16822663835936}
6 loss:  {'ner': 214.70782945850434}
7 loss:  {'ner': 197.1261684890578}
8 loss:  {'ner': 159.4629083336137}
9 loss:  {'ner': 140.97201797829868}
10 loss:  {'ner': 132.26447813196526}
11 loss:  {'ner': 112.6718197259706}
12 loss:  {'ner': 113.21399887730925}
13 loss:  {'ner': 105.7436702185181}
14 loss:  {'ner': 79.000099914653}
15 loss:  {'ner': 90.23178984944414}
16 loss:  {'ner': 71.79686657320269}
17 loss:  {'ner': 63.73600313695394}
18 loss:  {'ner': 72.97846881426537}
19 loss:  {'ner': 67.95532099762933}
20 loss:  {'ner': 51.744031716609776}
21 loss:  {'ner': 62.80218386691584}
22 loss:  {'ner': 68.60566623677165}
23 loss:  {'ner': 44.360469555585944}
24 loss:  {'ner': 49.511180488994924}
25 loss:  {'n

In [11]:
# Test the saved model
print("Loading from", output_dir)
nlp_test = spacy.load(output_dir)

Loading from /Users/ash/Desktop/custom_entity_extractor_spacy_change_direction_test


In [12]:
def del_from_list(del_arr, main_arr):
    for x in del_arr:
        if x in main_arr:
            main_arr.remove(x)
    return main_arr

In [22]:
def calc_res(user_ans, answers):    
    final_res = []
    grade_ent_res = {}
    tmp_del = []
    user = "machine"

    true_positive = 0
    false_negative = 0
    false_positive = 0
    complete = 0
    ent = "change_direction"
    grade_ent_res[user] = {}
    if ent not in grade_ent_res[user]:
        grade_ent_res[user][ent] = {"tp": 0, "fp": 0, "fn": 0, "complete": 0}

    if answers == user_ans:
        # 1. when both arrays are exactly the same
        for ans in user_ans:
            tmp_del.append(ans)
            true_positive += 1
            complete += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []
    else:
        # 2. when some elements are equal in both user and actual ans
        for ans in user_ans:
            if ans in answers:
                tmp_del.append(ans)
                true_positive += 1
                complete += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # 3. when user ans is partially correct
        # - if user ans is contained in actual ans
        if user_ans:
            for ans in user_ans:
                substring_user = [string for string in answers if ans in string]
                if substring_user:
                    tmp_del.append(ans)
                    true_positive += 1
                    false_positive += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # - if actual ans is contained in user ans
        if answers:
            for ans in answers:
                substring_ans = [string for string in user_ans if ans in string]
                if substring_ans:
                    tmp_del.append(ans)
                    true_positive += 1
                    false_positive += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # 4. when actual ans is missing from user ans
        if answers and user_ans:
            set_difference_actual = set(answers) - set(user_ans)
            for ans in list(set_difference_actual):
                tmp_del.append(ans)
                false_negative += 1
        answers = del_from_list(tmp_del, answers)
        tmp_del = []

        # 5. when user ans is missing from actual ans
        if answers and user_ans:
            set_difference_user = set(user_ans) - set(answers)
            for ans in list(set_difference_user):
                tmp_del.append(ans)
                false_positive += 1
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

    #print("tp: {}, fp: {}, fn: {}".format(true_positive, false_positive, false_negative))
    grade_ent_res[user][ent]["tp"] += true_positive
    grade_ent_res[user][ent]["fp"] += false_positive
    grade_ent_res[user][ent]["fn"] += false_negative
    grade_ent_res[user][ent]["complete"] += complete

    return grade_ent_res

In [32]:
result = os.path.join(output_dir, "spacy_result_change_dir.csv")
true_pos = 0
all_cd = 0
correct_cd = 0
false_pos = 0
false_neg = 0

headers = ["text", "predicted_cd", "actual_cd"]
final_res = []
final_res.append(headers)
for sent in TEST_DATA:
    doc2 = nlp_test(sent[0])
    res = [i.text for i in list(doc2.ents)]
    actual_cd = []
    for tokens in sent[1]['entities']:
        actual_cd.append(sent[0][tokens[0]:tokens[1]])
    final_res.append([sent[0], res, actual_cd])
    
with open(result, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(final_res)

print("created result file", result)

created result file /Users/ash/Desktop/custom_entity_extractor_spacy_change_direction_test/spacy_result_change_dir.csv


In [27]:
true_pos = 0
all_cd = 0
correct_cd = 0
false_pos = 0
false_neg = 0

for sent in TEST_DATA:
    doc2 = nlp_test(sent[0])
    res = [i.text for i in list(doc2.ents)]
    actual_cd = []
    for tokens in sent[1]['entities']:
        actual_cd.append(sent[0][tokens[0]:tokens[1]])
    all_cd += len(actual_cd)
    res_dict = calc_res(res, actual_cd)
    true_pos += res_dict["machine"]['change_direction']['tp']
    false_pos += res_dict["machine"]['change_direction']['fp']
    false_neg += res_dict["machine"]['change_direction']['fn']
    correct_cd += res_dict["machine"]['change_direction']['complete']
    
acc = ((correct_cd*100)/all_cd)
print("False pos: ", false_pos)
print("False neg: ", false_neg)
print("Marked {} out of {} cd correctly. Accuracy: {}".format(correct_cd, all_cd, acc))

False pos:  14
False neg:  6
Marked 61 out of 126 cd correctly. Accuracy: 48.41269841269841
